In [1]:
print("all ok")

all ok


In [2]:
import torch
import torchaudio
import librosa

e:\speech_recognition\speech_recognition\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
waveform, sample_rate = torchaudio.load("aryan_audio.mp3")

In [4]:
waveform

tensor([[-4.0349e-06, -4.4761e-06, -1.0259e-05,  ...,  9.8359e-04,
          2.4207e-03,  3.2772e-03]])

In [6]:
len(waveform[0])

288856

In [8]:
sample_rate

#A sample rate is the number of audio measurements taken per second when converting sound into digital form.

48000

In [10]:
waveform = librosa.resample(waveform, sample_rate)

TypeError: resample() takes 1 positional argument but 2 were given

In [1]:
import sys
import os
import torch
from model import VoiceRecognitionModel
from train import load_and_preprocess_data, train_model
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

e:\speech_recognition\speech_recognition\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\speech_recognition\speech_recognition\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


ImportError: cannot import name 'load_and_preprocess_data' from 'train' (e:\speech_recognition\train.py)

In [2]:
import librosa
import numpy as np

def preprocess_voice(file_path):
    # Load the audio file
    audio, sr = librosa.load(file_path, sr=None)
    # Extract features like MFCC
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)


In [3]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
import torch

class VoiceRecognitionModel:
    def __init__(self):
        self.processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        self.model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-base-960h")

    def train(self, train_data, labels):
        # Prepare input tensors
        inputs = self.processor(train_data, sampling_rate=16000, return_tensors="pt", padding=True)
        labels = torch.tensor(labels).unsqueeze(0)
        # Training process (simplified)
        self.model.train()
        outputs = self.model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()

    def predict(self, audio):
        inputs = self.processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
        logits = self.model(**inputs).logits
        predicted_id = torch.argmax(logits, dim=-1).item()
        return predicted_id


In [9]:
import librosa
import numpy as np

def preprocess_voice(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)

# Example: Process your three voice samples
voice1_features = preprocess_voice('nirupam_audio.mp3')
voice2_features = preprocess_voice('aryan_audio.mp3')
voice3_features = preprocess_voice('ashutosh_audio.mp3')

# Prepare the training data
train_data = np.array([voice1_features, voice2_features, voice3_features])
labels = np.array([0, 1, 2])  # Assign unique labels to each voice


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleVoiceRecognitionModel(nn.Module):
    def __init__(self):
        super(SimpleVoiceRecognitionModel, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)  # Output layer for 3 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleVoiceRecognitionModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert data to tensors
train_data_tensor = torch.tensor(train_data, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Training loop
for epoch in range(100):  # Train for 100 epochs
    optimizer.zero_grad()
    outputs = model(train_data_tensor)
    loss = criterion(outputs, labels_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f'Epoch [{epoch}/100], Loss: {loss.item():.4f}')


Epoch [0/100], Loss: 8.4663
Epoch [20/100], Loss: 0.1742
Epoch [40/100], Loss: 0.0734
Epoch [60/100], Loss: 0.0416
Epoch [80/100], Loss: 0.0266


In [12]:
def predict(model, file_path):
    features = preprocess_voice(file_path)
    features_tensor = torch.tensor(features, dtype=torch.float32)
    with torch.no_grad():
        output = model(features_tensor)
    predicted_label = torch.argmax(output).item()
    return predicted_label

# Example usage
predicted_label = predict(model, 'testing_audio.mp3')
print(f'Predicted Label: {predicted_label}')


Predicted Label: 1


In [13]:
import streamlit as st

st.title("Voice Recognition System")

uploaded_file = st.file_uploader("Upload a voice sample", type=["wav", "mp3"])

if uploaded_file is not None:
    st.audio(uploaded_file, format="audio/wav")
    predicted_label = predict(model, uploaded_file)
    if predicted_label == 0  :  # Adjust according to your labels
        st.success("Unlocked!")
    else:
        st.error("Access Denied.")


2024-08-26 21:19:59.668 
  command:

    streamlit run e:\speech_recognition\speech_recognition\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
